In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/united-hackathon-dataset/sentiment_statisticscc1e57a.csv
/kaggle/input/united-hackathon-dataset/customers2afd6ea.csv
/kaggle/input/united-hackathon-dataset/testbc7185d.csv
/kaggle/input/united-hackathon-dataset/callsf0d4f5a.csv
/kaggle/input/united-hackathon-dataset/reason18315ff.csv


In [16]:
import pandas as pd

In [17]:
calls_df = pd.read_csv('/kaggle/input/united-hackathon-dataset/callsf0d4f5a.csv')
customers_df = pd.read_csv('/kaggle/input/united-hackathon-dataset/customers2afd6ea.csv')
reason_df = pd.read_csv('/kaggle/input/united-hackathon-dataset/reason18315ff.csv')
sentiment_df = pd.read_csv('/kaggle/input/united-hackathon-dataset/sentiment_statisticscc1e57a.csv')

In [18]:
calls_df.head(5)


,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript
0,4667960400,2033123310,963118,7/31/2024 23:56,8/1/2024 0:03,8/1/2024 0:34,\n\nAgent: Thank you for calling United Airlin...
1,1122072124,8186702651,519057,8/1/2024 0:03,8/1/2024 0:06,8/1/2024 0:18,\n\nAgent: Thank you for calling United Airlin...
2,6834291559,2416856629,158319,7/31/2024 23:59,8/1/2024 0:07,8/1/2024 0:26,\n\nAgent: Thank you for calling United Airlin...
3,2266439882,1154544516,488324,8/1/2024 0:05,8/1/2024 0:10,8/1/2024 0:17,\n\nAgent: Thank you for calling United Airlin...
4,1211603231,5214456437,721730,8/1/2024 0:04,8/1/2024 0:14,8/1/2024 0:23,\n\nAgent: Thank you for calling United Airlin...


In [19]:
customers_df.head(5)


,customer_id,customer_name,elite_level_code
0,2033123310,Matthew Foster,4.0
1,8186702651,Tammy Walters,NaN
2,2416856629,Jeffery Dixon,NaN
3,1154544516,David Wilkins,2.0
4,5214456437,Elizabeth Daniels,0.0


In [20]:
reason_df.head(5)


,call_id,primary_call_reason
0,4667960400,Voluntary Cancel
1,1122072124,Booking
2,6834291559,IRROPS
3,2266439882,Upgrade
4,1211603231,Seating


In [21]:
sentiment_df.head(5)

,call_id,agent_id,agent_tone,customer_tone,average_sentiment,silence_percent_average
0,4667960400,963118,neutral,angry,-0.04,0.39
1,1122072124,519057,calm,neutral,0.02,0.35
2,6834291559,158319,neutral,polite,-0.13,0.32
3,2266439882,488324,neutral,frustrated,-0.20,0.20
4,1211603231,721730,neutral,polite,-0.05,0.35


In [22]:
calls_df['call_start_datetime'] = pd.to_datetime(calls_df['call_start_datetime'])
calls_df['agent_assigned_datetime'] = pd.to_datetime(calls_df['agent_assigned_datetime'])
calls_df['call_end_datetime'] = pd.to_datetime(calls_df['call_end_datetime'])

In [23]:
calls_df['AHT'] = (calls_df['call_end_datetime'] - calls_df['agent_assigned_datetime']).dt.total_seconds()

calls_df['AST'] = (calls_df['agent_assigned_datetime'] - calls_df['call_start_datetime']).dt.total_seconds()

calls_df[['call_id', 'AHT', 'AST']].head()

,call_id,AHT,AST
0,4667960400,1860.0,420.0
1,1122072124,720.0,180.0
2,6834291559,1140.0,480.0
3,2266439882,420.0,300.0
4,1211603231,540.0,600.0


In [24]:
merged_df = pd.merge(calls_df, sentiment_df, on='call_id')

In [25]:
sentiment_AHT_AST = merged_df.groupby('customer_tone').agg(
    avg_AHT=('AHT', 'mean'),
    avg_AST=('AST', 'mean'),
    avg_sentiment=('average_sentiment', 'mean'),
    count=('call_id', 'count')
).reset_index()

sentiment_AHT_AST

,customer_tone,avg_AHT,avg_AST,avg_sentiment,count
0,angry,695.530261,436.445031,-0.032427,14408
1,calm,699.604098,437.832299,-0.032501,14347
2,frustrated,692.738764,436.803371,-0.034490,14240
3,neutral,707.625312,437.754085,-0.033709,14444
4,polite,689.659731,436.499896,-0.034475,14371


In [26]:
calls_df['call_hour'] = calls_df['call_start_datetime'].dt.hour


aht_ast_by_hour = calls_df.groupby('call_hour').agg(
    avg_AHT=('AHT', 'mean'),
    avg_AST=('AST', 'mean'),
    call_count=('call_id', 'count')
).reset_index()

aht_ast_by_hour.sort_values(by='call_hour', ascending=True)

,call_hour,avg_AHT,avg_AST,call_count
0,0,682.545772,438.831735,1147
1,1,757.087034,432.362345,1126
2,2,703.973384,436.825095,1052
3,3,706.529467,440.823199,1069
4,4,688.331762,430.235627,1061
5,5,691.630435,438.423913,1104
6,6,698.277778,431.111111,1080
7,7,698.502475,450.259901,1616
8,8,703.986916,440.781220,5197
9,9,712.512686,438.616801,5321


In [27]:
def classify_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

calls_df['time_of_day'] = calls_df['call_hour'].apply(classify_time_of_day)

traffic_by_time_of_day = calls_df.groupby('time_of_day').agg(
    call_count=('call_id', 'count'),
    avg_AHT=('AHT', 'mean'),
    avg_AST=('AST', 'mean')
).reset_index()

traffic_by_time_of_day.sort_values(by='call_count', ascending=False)

,time_of_day,call_count,avg_AHT,avg_AST
0,Afternoon,31312,693.543051,436.862545
2,Morning,23657,698.198419,438.479097
1,Evening,10282,699.908578,434.956234
3,Night,6559,705.153225,436.264674


In [28]:
# reason_df = pd.read_csv('/mnt/data/reason18315ff.csv')

# Get the distinct values in the 'primary_call_reason' column
distinct_primary_reasons = reason_df['primary_call_reason'].unique()

distinct_primary_reasons

array(['Voluntary  Cancel', 'Booking', 'IRROPS', 'Upgrade', 'Seating',
       'Mileage   Plus', 'Checkout', 'Voluntary   Change', 'Post Flight',
       'Check In', 'Other  Topics', 'Communications', 'Schedule Change',
       'Products & Services', 'IRROPS  ', 'Digital   Support',
       'Seating  ', 'Disability', 'Unaccompanied Minor', ' Baggage',
       'Traveler  Updates', 'Communications  ', 'ETC', 'Upgrade  ',
       'Unaccompanied Minor  ', 'Voluntary Change', 'Voluntary Change  ',
       'Checkout  ', 'Mileage Plus', 'Mileage Plus  ', 'Booking  ',
       'Baggage  ', 'Post-Flight', 'Post-Flight  ', 'Schedule Change  ',
       'Baggage', 'Traveler Updates', 'Voluntary Cancel', 'Check-In',
       'Products and Services', 'Check-In  ', 'Other Topics',
       'Other Topics  ', 'ETC  ', 'Disability  ', 'Digital Support',
       'Digital Support  ', 'Voluntary Cancel  ',
       'Products and Services  ', 'Traveler Updates  ',
       'Traveler   Updates', 'Digital  Support', 'Mileage  P